# EDS296: Final Mini-Project
### **Authors**

Carmen Hoyt choyt@ucsb.edu

Ryan Green **EMAIL**

Rachel Swick **EMAIL**

### [GitHub Repository](https://github.com/ceh58/eds296-final)

## Introductcion

1. Choose a spatial region you find interesting, for whatever reason.

Write a few paragraphs about this place! Include a relevant graphic or two (can be an
infographic or map from the Internet) showing the geographical and climatological context
of your region. What’s going on there? What types of extreme events are relevant? How
might you expect that climate change impacts could be felt by human or natural systems?

**Region**: Indonesia

**Bounding box**: [92, 146, -15, 10]

**Climate**: 

Indonesia has a tropical climate, experiencing two seasons: wet (November-April) and dry (May-October).  It is a key part of the Coral Triangle, a region with at least 500 species of important reef-building corals. Corals are highly sensitive to changes in water temperature, and will bleach (expel their zooxanthellae) after prolonged exposure to warmer temperatrues (or changes of greater than 1°C). I would expect climate change to drive ocean temperatures up, leading to widespread bleaching and mass coral loss. Coral loss has many implications for the oceans and the people that depend on them, such as loss in food source, coastal protection, and tourism. 

Indonesia's first mass bleacing event was recorded in [1997-1998](https://indonesia.wcs.org/Initiatives/Marine/Coral-Bleaching.aspx). Since then, there have also been bleaching events in [2016-2017 and 2021](https://theseapeople.org/2024/12/18/coral-bleaching-raja-ampat/). 


Thus, I am interested in looking at historical (1900-1930) and late century (1995-2025) average sea surface temperatures. I chose the late 20th century time period to cover the span of the mass bleaching events to see if there was a significant change (increase) in ocean temperature as compared to the historical temperature. 

**Write about how affected by el nino/la nina!!**

## Metrics Definition

2. Define metrics you plan to use to assess climate change impacts in your region.

Given the context you just provided in item 1, decide on two different ‘metrics’, or quantities
to be calculated from climate model output, that will help you quantify climate change
impacts. Some examples of useful metrics might be:

- Regional averages (to let you plot time series)
- Time averages (to let you plot maps)
- Correlation patterns

You’ll also need to decide on a set of variables to include, from which to calculate the
metrics above. This could be temperature, precipitation, wind, or anything else you can find
that you think is helpful

**Metrics**:

1. Regional averages

This will allow us to quantify climate change impacts over time regarding changes in 'pr' and 'tos'. Are they seeing similar trends? 

2. Correlation patterns

Regression: How quickly are changes in 'pr' and 'tos' happening (rate of change)?

**Variables**:

- "pr"

Since Indonesia has wet and dry seasons, precipitation can be an important indicator for climate change impacts.

- "tos"

Additionally, much of Indonesia's economy is reliant upon the ocean. Changes in sea surface temperature could have detrimental effects.


## Data I/O

3. Extract necessary climate (historical plus projection) data for that region.

Write code to extract the variables you need to do your calculations, and to store these in
local memory in a way that makes it easy to compute the metrics you’ve chosen. For
instance: generating an xarray object containing all ensemble members for your chosen set of
climate models!
Note: I recommend choosing a subset of climate models and scenarios to use for this
analysis, since you likely won’t be able to look at everything. Picking something like 2-3
different climate models and 1-2 different scenarios to work with should let you have a good
range of estimates, without putting TOO much demand on your laptop’s memory!


**Climate models**:

- CESM2

**ADD DESCRIPTION**

- ACCESS-CM2

The [ACCESS-CM2 (Australian Community Climate and Earth-System Simulator Coupled Model version 2)](https://research.csiro.au/access/about/cm2/) is a climate model developed by the Australian Bureau of Meteorology and CSIRO for participation in the CMIP6 (Coupled Model Intercomparison Project Phase 6). Ocean variables are modeled using MOM5 (Modular Ocean Model version 5) at 1 x 1 degree resolution.

**Scenarios**:

- SSP370: 'Regional Rivalry' 

The SSP3-7.0 scenario (Shared Socioeconomic Pathway 3 with a radiative forcing of 7.0 W/m² by 2100) is one of the future climate projections used in CMIP6. It represents a high-emissions pathway, with CO² emissions projected to double by 2100. This can lead to significant challenges in both mitigation and adaptation.

- SSP585: 'Fossil-Fueled Development'

The SSP5-8.5 scenario (Shared Socioeconomic Pathway 5 with a radiative forcing of 8.5 W/m² by 2100) is the highest-emissions pathway in the CMIP6 framework and represents a fossil-fueled development future. It predicts very high GHG emissions, with CO2 emissions tripling by 2075. This scenario provides insight into the upper range of climate system response to greenhouse gas forcing, useful for stress-testing infrastructure, understanding tipping points, and preparing for high-impact outcomes.


### Import Packages

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import intake
import s3fs
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.stats as stats

ERROR 1: PROJ: proj_create_from_database: Open of /opt/anaconda3/envs/eds296-stevenson/share/proj failed


### Access the CMIP6 Database

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

#### CESM2 Model

##### Search Catalog

In [14]:
# Specify search terms to query catalog 
# activity_id: which downscaling technique do you want?
activity_id = ["CMIP"]

# experiment_id: which historical/future scenario do you want?
experiment_id = ["historical", "ssp370"]

# table_id: which part of the Earth system/time resolution do you want?
table_id = ["Amon", "Omon"] # both atmos and ocaen

# source_id: which model do you want?
source_id = ["CESM2"]

# variable_id: which variable do you want?
variable_id = ["tos", "pr"]

##### Store 'tos'

In [8]:
# Search through catalog, store results for tos
cesm_res_tos = catalog.search(activity_id=activity_id, experiment_id=experiment_id,
                         source_id=source_id, table_id=table_id, 
                         variable_id='tos', grid_label='gr') # remove gn

# Recreate the data frame
cesm_tos_df = cesm_res_tos.df

# Display the data frame
cesm_tos_df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2,historical,r3i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
1,CMIP,NCAR,CESM2,historical,r5i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
2,CMIP,NCAR,CESM2,historical,r2i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
3,CMIP,NCAR,CESM2,historical,r1i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
4,CMIP,NCAR,CESM2,historical,r6i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
5,CMIP,NCAR,CESM2,historical,r4i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190308
6,CMIP,NCAR,CESM2,historical,r7i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190311
7,CMIP,NCAR,CESM2,historical,r8i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190311
8,CMIP,NCAR,CESM2,historical,r9i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190311
9,CMIP,NCAR,CESM2,historical,r10i1p1f1,Omon,tos,gr,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190313


##### Store 'pr'

In [10]:
# Search through catalog, store results for precip
cesm_res_pr = catalog.search(activity_id=activity_id, experiment_id=experiment_id,
                         source_id=source_id, table_id=table_id, 
                         variable_id='pr')

# Recreate the data frame
cesm_pr_df = cesm_res_pr.df

# Display the data frame
cesm_pr_df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2,historical,r5i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
1,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
2,CMIP,NCAR,CESM2,historical,r8i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
3,CMIP,NCAR,CESM2,historical,r4i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
4,CMIP,NCAR,CESM2,historical,r2i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
5,CMIP,NCAR,CESM2,historical,r3i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
6,CMIP,NCAR,CESM2,historical,r7i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
7,CMIP,NCAR,CESM2,historical,r6i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190401
8,CMIP,NCAR,CESM2,historical,r11i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190514
9,CMIP,NCAR,CESM2,historical,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20200124


In [11]:
# Define list of ensemble members
# tos
mems_tos = cesm_tos_df.member_id.unique()

# pr
mems_pr = cesm_pr_df.member_id.unique()

# Find the ones each list has in common
common_mems = list(set(mems_tos) & set(mems_pr))

# Print
print(common_mems)

['r6i1p1f1', 'r10i1p1f1', 'r8i1p1f1', 'r5i1p1f1', 'r3i1p1f1', 'r11i1p1f1', 'r1i1p1f1', 'r7i1p1f1', 'r4i1p1f1', 'r9i1p1f1', 'r2i1p1f1']


#### ACCESS-CM2 Model

##### Specify Model

In [23]:
# source_id: which model do you want?
source_id = ["ACCESS-CM2"]

# THIS IS NOT RETURNING ANYTHING

##### Store 'tos'

In [24]:
# Search through catalog, store results for tos
access_res_tos = catalog.search(activity_id=activity_id, experiment_id=experiment_id,
                         source_id=source_id, table_id=table_id, 
                         variable_id='tos', grid_label='gr') # remove gn

# Recreate the data frame
access_tos_df = access_res_tos.df

# Display the data frame
display(access_tos_df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version


##### Store 'pr'

In [ ]:
# Search through catalog, store results for precip
access_res_pr = catalog.search(activity_id=activity_id, experiment_id=experiment_id,
                         source_id=source_id, table_id=table_id, 
                         variable_id='pr')

# Recreate the data frame
access_pr_df = access_res_pr.df

# Display the data frame
display(access_pr_df)

##### Store Results

In [ ]:
# Search through catalog, store results
access_res = catalog.search(activity_id=activity_id, experiment_id=experiment_id,
                         source_id=source_id, table_id=table_id, variable_id=variable_id)

# Create a data frame
access_df = access_res.df

# Look at results
display(access_df)

## Metrics Computation

4. Compute metrics based on extracted data.

Now write additional code to take the set of variables you just extracted, and calculating the
necessary metrics (averages, etc) from them.

## Visualizations

5. Create visualizations of climate change impacts.

Make some plots showing your metrics and how they change as a function of time!

## Conclusions

6. Interpret and contextualize your results.

Finally, wrap it all up with a couple of paragraphs discussing your results and what you think
their implications might be for your region. For instance: what populations or ecosystems
might be impacted? How confident are you that the future changes are different from the
historical climate? Is there anything else you’re excited to talk about?